In [1]:
import pandas as pd
from datetime import datetime as dt
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import json



In [4]:
df = pd.read_csv("../Data/Crimes2001_to_Present.csv")

In [5]:
df["Month"] = df["Date"].apply(lambda x: dt.strptime(x, "%m/%d/%Y %I:%M:%S %p").month)

In [5]:
years = [i for i in range(2022, 2024)]
months = [i for i in range(1, 13)]
for year in years:
    for month in months:
       partial_df = df[(df["Year"] == year) & (df["Month"] == month)].copy()
       partial_df.dropna(inplace=True)  # Modify the copied DataFrame
       data_dict = partial_df.to_dict(orient='split')["data"]
       ref = db.reference(f"/{year}/{month}/")
       ref.set(data_dict)

In [6]:
#creatying one big json file
partial_df = df[(df["Year"] >= 2017) & (df["Year"] <= 2023)].copy()
partial_df.dropna(inplace=True)  # Modify the copied DataFrame
partial_df.to_json("../Data/2017-2023.json", orient='split')

In [8]:
#get violent crime counts by ward and 
partial_df = df[(df["Year"] >= 2017) & (df["Year"] <= 2023)].copy()
violentCrimes = ['BATTERY', 'ASSAULT', 'ROBBERY', 'CRIMINAL SEXUAL ASSAULT', 'HOMICIDE']

ward_data = {

}
wards = [i + 1 for i in range(50)]
years = [i for i in range(2017, 2024)]
for ward in wards:
    data = {}
    for year in years: 
        filtered_partial_df = partial_df[(partial_df["Ward"] == ward) & (partial_df["Year"] == year) & (partial_df["Primary Type"].isin(violentCrimes))]
        data[year] = {
            "crimeCount": len(filtered_partial_df)
        }
    ward_data[ward] = data

print(ward_data)


{1: {2017: {'crimeCount': 1118}, 2018: {'crimeCount': 998}, 2019: {'crimeCount': 1119}, 2020: {'crimeCount': 840}, 2021: {'crimeCount': 949}, 2022: {'crimeCount': 1053}, 2023: {'crimeCount': 1142}}, 2: {2017: {'crimeCount': 3118}, 2018: {'crimeCount': 2479}, 2019: {'crimeCount': 1043}, 2020: {'crimeCount': 687}, 2021: {'crimeCount': 920}, 2022: {'crimeCount': 869}, 2023: {'crimeCount': 937}}, 3: {2017: {'crimeCount': 2606}, 2018: {'crimeCount': 2642}, 2019: {'crimeCount': 2548}, 2020: {'crimeCount': 2143}, 2021: {'crimeCount': 2282}, 2022: {'crimeCount': 2303}, 2023: {'crimeCount': 2311}}, 4: {2017: {'crimeCount': 1530}, 2018: {'crimeCount': 1648}, 2019: {'crimeCount': 2086}, 2020: {'crimeCount': 1814}, 2021: {'crimeCount': 1963}, 2022: {'crimeCount': 2267}, 2023: {'crimeCount': 2297}}, 5: {2017: {'crimeCount': 2272}, 2018: {'crimeCount': 2117}, 2019: {'crimeCount': 1991}, 2020: {'crimeCount': 1854}, 2021: {'crimeCount': 1936}, 2022: {'crimeCount': 1931}, 2023: {'crimeCount': 1912}}, 6

In [10]:
import json  
json_object = json.dumps(ward_data, indent = 4) 
file_name = "../Data/ward_data.json"

with open(file_name, 'w') as file:
    file.write(json_object)

